# 20241130 
### ADI, OCO를 통합하고자 만듬
### ※주의※  UNIQUE_ID 부분에  E1,E2,E3가 들어가는데, OCO기준(1~80/81~160/161~240)으로만 되어있어서.  ADI 멀티타켓은 대응되지않음. 

In [ ]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl
import time
from datetime import datetime




################################### K MRC fit & Decorrect #####################################################################


def kmrc_decorrect(df_rawdata, df_mrc_input):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID3')   # UNIQUE_ID는  E1,E2,E3가 붙어서 MRC랑 매칭이 안됨. ! 

    # 예측 결과를 저장할 리스트
    mrc_list = []

    for unique_id, group in grouped:
        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordinate_y = group['coordinate_Y']

      
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = coordinate_x
        ry = coordinate_y

        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 현재 UNIQUE_ID에 해당하는 MRC 데이터만 선택
        mrc_k_odd = df_mrc_input[
            (df_mrc_input['UNIQUE_ID'] == unique_id) & 
            (df_mrc_input['K PARA'].isin(['W1', 'W3', 'W5', 'W7', 'W9', 'W11', 'W13', 'W15', 'W17', 'W19', 
                                          'R3', 'R5', 'R7', 'R9', 'R11', 'R13', 'R15', 'R17', 'R19']))
        ]

        mrc_k_even = df_mrc_input[
            (df_mrc_input['UNIQUE_ID'] == unique_id) & 
            (df_mrc_input['K PARA'].isin(['W2', 'W4', 'W6', 'W8', 'W10', 'W12', 'W14', 'W16', 'W18', 'W20', 
                                          'R4', 'R6', 'R8', 'R10', 'R12', 'R14', 'R16', 'R18']))
        ]

        # 'K PARA'가 정확히 19개씩 있는지 확인
        if len(mrc_k_odd) != 19 or len(mrc_k_even) != 18:
            print(f"UNIQUE_ID {unique_id}의 MRC 데이터가 부족합니다.")
            continue

        # 'GPM' 값을 numpy 배열로 변환
        mrc_k_odd_values = mrc_k_odd['GPM'].values.astype(float)
        mrc_k_even_values = mrc_k_even['GPM'].values.astype(float)



        # mrc fitting (부호반대처리)
        mrc_fit_x = X_dx.dot(mrc_k_odd_values) * -1
        mrc_fit_y = X_dy.dot(mrc_k_even_values) * -1

        # X_reg 에서 kmrc 빼주기
        X_reg_demrc = group['X_reg'] - mrc_fit_x
        Y_reg_demrc = group['Y_reg'] - mrc_fit_y

        # 결과 저장
        mrc_list.append(pd.DataFrame({              
            'mrc_fit_x': mrc_fit_x,
            'mrc_fit_y': mrc_fit_y,
            'X_reg_demrc': X_reg_demrc,
            'Y_reg_demrc': Y_reg_demrc,
        }))

    # mrc 결과 병합
    df_mrc_de = pd.concat(mrc_list, ignore_index=True)
    return df_mrc_de






###################### PSM Input은 빼주고 Point MRC는 더해준게 MRC_X임.   X_reg_demrc + MRC X ###########################


def remove_psm_add_pointmrc(df_rawdata):
    df_rawdata = pd.DataFrame(df_rawdata)
    df_rawdata['raw_x'] = df_rawdata['X_reg_demrc'] + df_rawdata['MRC_X'] 
    df_rawdata['raw_y'] = df_rawdata['Y_reg_demrc'] + df_rawdata['MRC_Y'] 
    return df_rawdata







###################### OSR ###########################

def multi_lot_regression(df_rawdata):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')

    # 회귀분석 결과를 저장할 리스트
    wkrk_results = []

    # 각 그룹에 대해 처리
    for unique_id, group in grouped:
        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        field_x = group['coordinate_X']
        field_y = group['coordinate_Y']
        
        # 좌표 계산
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = field_x
        ry = field_y

        # X_dx, X_dy 행렬 구성
        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 종속변수
        Y_dx = group['X_reg']
        Y_dy = group['Y_reg']

        # 최소자승법으로 계수 계산
        coeff_dx = np.linalg.lstsq(X_dx, Y_dx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Y_dy, rcond=None)[0]

        # 결과 저장
        wkrk_results.append(pd.DataFrame({
            'UNIQUE_ID': [unique_id],
            'WK1': [coeff_dx[0]],
            'WK2': [coeff_dy[0]],
            'WK3': [coeff_dx[1]],
            'WK4': [coeff_dy[1]],
            'WK5': [coeff_dx[2]],
            'WK6': [coeff_dy[2]],
            'WK7': [coeff_dx[3]],
            'WK8': [coeff_dy[3]],
            'WK9': [coeff_dx[4]],
            'WK10': [coeff_dy[4]],
            'WK11': [coeff_dx[5]],
            'WK12': [coeff_dy[5]],
            'WK13': [coeff_dx[6]],
            'WK14': [coeff_dy[6]],
            'WK15': [coeff_dx[7]],
            'WK16': [coeff_dy[7]],
            'WK17': [coeff_dx[8]],
            'WK18': [coeff_dy[8]],
            'WK19': [coeff_dx[9]],
            'WK20': [coeff_dy[9]],
            'RK1': [0],
            'RK2': [0],
            'RK3': [coeff_dx[10]],
            'RK4': [coeff_dy[10]],
            'RK5': [coeff_dx[11]],
            'RK6': [coeff_dy[11]],
            'RK7': [coeff_dx[12]],
            'RK8': [coeff_dy[12]],
            'RK9': [coeff_dx[13]],
            'RK10': [coeff_dy[13]],
            'RK11': [coeff_dx[14]],
            'RK12': [coeff_dy[14]],
            'RK13': [coeff_dx[15]],
            'RK14': [coeff_dy[15]],
            'RK15': [coeff_dx[16]],
            'RK16': [coeff_dy[16]],
            'RK17': [coeff_dx[17]],
            'RK18': [coeff_dy[17]],
            'RK19': [coeff_dx[18]],
            'RK20': [0],
        }))

    # 결과 병합
    combined_results = pd.concat(wkrk_results, ignore_index=True)
    return combined_results



###################### OSR Fitting, Residual ###########################


def multi_lot_fitting_residual(df_rawdata, df_coeff):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')
    
    # 예측 결과를 저장할 리스트
    predictions_list = []
    
    for unique_id, group in grouped:
 

        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordiante_y = group['coordinate_Y']   
              
        
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = coordinate_x
        ry = coordiante_y

        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 해당 LOT_ID의 계수 추출
        coeffs = df_coeff[df_coeff['UNIQUE_ID'] == unique_id].iloc[0]
        coeff_dx = coeffs[['WK1','WK3','WK5','WK7','WK9','WK11','WK13','WK15','WK17','WK19','RK3','RK5','RK7','RK9','RK11','RK13','RK15','RK17','RK19']].values.astype(float)
        coeff_dy = coeffs[['WK2','WK4','WK6','WK8','WK10','WK12','WK14','WK16','WK18','WK20','RK4','RK6','RK8','RK10','RK12','RK14','RK16','RK18']].values.astype(float)


        # 예측값 계산
        pred_x = X_dx.dot(coeff_dx)
        pred_y = X_dy.dot(coeff_dy)

        # 잔차 계산
        residual_x = group['X_reg'] - pred_x
        residual_y = group['Y_reg'] - pred_y

        # 결과 저장
        predictions_list.append(pd.DataFrame({
            'pred_x': pred_x,
            'pred_y': pred_y,
            'residual_x': residual_x,
            'residual_y': residual_y,
        }))

    # 예측 결과 병합
    df_predictions = pd.concat(predictions_list, ignore_index=True)
    return df_predictions







###################### PSM Input Decorrect ################################ 

def psm_decorrect(df_rawdata, df_psm_input):
    # 'shot' 별로 데이터를 그룹화 (고유한 lot_id, die_x, die_y 조합)
    grouped = df_rawdata.groupby(['UNIQUE_ID3', 'DieX', 'DieY'])

    # 예측 결과를 저장할 리스트
    psm_input_list = []

    # 각 그룹에 대해 연산 수행
    for (unique_id, diex, diey), group in grouped: 
        test = group['TEST']
        die_x = group['DieX']
        die_y = group['DieY']
               
       
        # 독립변수 설정 ('coordinate_X', 'coordinate_Y'를 독립변수로 사용)
        rx = group['coordinate_X'].values
        ry = group['coordinate_Y'].values

        # 독립 변수 배열구성
        X_dx = np.vstack([
            np.ones(len(rx)),
            (rx)/1e6,     (-ry)/1e6, 
            (rx**2)/1e9,  (rx*ry)/1e9,     (ry**2)/1e9,
            (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12,    (ry**3)/1e12,
            (rx**4)/1e19, (rx**3*ry)/1e19, (rx**2*ry**2)/1e19, (rx*ry**3)/1e19,    (ry**4)/1e19,
            (rx**5)/1e23, (rx**4*ry)/1e23, (rx**3*ry**2)/1e23, (rx**2*ry**3)/1e23, (rx*ry**4)/1e23,    (ry**5)/1e23,
            (rx**6)/1e27, (rx**5*ry)/1e27, (rx**4*ry**2)/1e27, (rx**3*ry**3)/1e27, (rx**2*ry**4)/1e27, (rx*ry**5)/1e27,    (ry**6)/1e27,
            (rx**7)/1e31, (rx**6*ry)/1e31, (rx**5*ry**2)/1e31, (rx**4*ry**3)/1e31, (rx**3*ry**4)/1e31, (rx**2*ry**8)/1e31, (rx*ry**6)/1e31, (ry**7)/1e31
        ]).T

        X_dy = np.vstack([
            np.ones(len(ry)), 
            (ry)/1e6,     (rx)/1e6,
            (ry**2)/1e9,  (ry*rx)/1e9,     (rx**2)/1e9,
            (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12,    (rx**3)/1e12,
            (ry**4)/1e19, (ry**3*rx)/1e19, (ry**2*rx**2)/1e19, (ry*rx**3)/1e19,    (rx**4)/1e19,
            (ry**5)/1e23, (ry**4*rx)/1e23, (ry**3*rx**2)/1e23, (ry**2*rx**3)/1e23, (ry*rx**4)/1e23,    (rx**5)/1e23,
            (ry**6)/1e27, (ry**5*rx)/1e27, (ry**4*rx**2)/1e27, (ry**3*rx**3)/1e27, (ry**2*rx**4)/1e27, (ry*rx**5)/1e27,    (rx**6)/1e27,
            (ry**7)/1e31, (ry**6*rx)/1e31, (ry**5*rx**2)/1e31, (ry**4*rx**3)/1e31, (ry**3*rx**4)/1e31, (ry**2*rx**8)/1e31, (ry*rx**6)/1e31, (rx**7)/1e31             
        ]).T


        # 종속변수 설정 ( PSM INPUT의 RK값을 독립변수로 사용)
        # PerShotMRC 시트에서 해당 UNIQUE_ID에 해당하는 rk1~rk72 값을 추출
        psm_row = df_psm_input[(df_psm_input['UNIQUE_ID'] == unique_id) & (df_psm_input['dCol'] == diex) & (df_psm_input['dRow'] == diey)]

        if psm_row.empty:
            # PSM input이 없는 경우 0으로 처리
            Y_dx = np.zeros(36)  # 홀수 rk값의 수만큼 0 배열 생성
            Y_dy = np.zeros(36)  # 짝수 rk값의 수만큼 0 배열 생성
        else:
            # 홀수 rk 값 (Y_dx)과 짝수 rk 값 (Y_dy) 추출
            rk_values = psm_row.iloc[:, 15:87]  # rk1 ~ rk72 열을 선택    # MMO_MRC_EQP 신규컬럼 추가되면서 15:87 -> 16:88로 변경함 -> 15:87로 다시 변경(2024-10-10 04:26)

            # Y_dx는 rk1부터 rk71까지 홀수 열을 추출
            Y_dx = rk_values.iloc[:, ::2].values.flatten()       

            # Y_dy는 rk2부터 rk72까지 짝수 열을 추출
            Y_dy = rk_values.iloc[:, 1::2].values.flatten()  


        # 행렬 곱을 통해 예측 값 계산
        psm_fit_x = X_dx.dot(Y_dx)
        psm_fit_y = X_dy.dot(Y_dy)       

        residual_x_depsm = group['residual_x'] - psm_fit_x
        residual_y_depsm = group['residual_y'] - psm_fit_y

        # 결과 저장
        psm_input_list.append(pd.DataFrame({
            'UNIQUE_ID': unique_id,
            'TEST' : test,
            'DieX' : die_x,
            'DieY' : die_y,
            'psm_fit_x': psm_fit_x,
            'psm_fit_y': psm_fit_y,
            'residual_x_depsm' : residual_x_depsm,
            'residual_y_depsm' : residual_y_depsm
               
        }))


    
    # 결과 병합
    df_psm_de = pd.concat(psm_input_list, ignore_index=True)

    # 정렬 (★ 기존데이터의 정렬순서와 맞춰주기위한 작업)
    df_psm_de = df_psm_de.sort_values(by=['UNIQUE_ID', 'TEST', 'DieX', 'DieY'])

    # 'UNIQUE_ID', 'TEST', 'DieX', 'DieY' 컬럼을 삭제
    df_psm_de = df_psm_de.drop(['UNIQUE_ID', 'TEST', 'DieX', 'DieY'], axis=1)

    return df_psm_de






###################### resi_depsm 을 CPE 19para 모델링 ################################ 



def resi_to_cpe(df_rawdata):
    # 'shot' 별로 데이터를 그룹화 (고유한 die_x, die_y 조합)
    grouped = df_rawdata.groupby(['UNIQUE_ID', 'DieX', 'DieY'])
    
    # 회귀분석 결과를 저장할 리스트
    shot_regression_results = []

    for (unique_id, die_x, die_y), group in grouped:
        # 독립변수와 종속변수 설정 
        # 독립변수 (shot 좌표)
        rx = group['coordinate_X']
        ry = group['coordinate_Y']  

        # 종속변수 (residual)
        Yx = group['residual_x_depsm']
        Yy = group['residual_y_depsm']



        # 독립 변수 배열구성 ( 19PARA    X : 1023 // Y : 511  )
        X_dx = np.vstack([
            np.ones(len(rx)),
            (rx)/1e6,     (-ry)/1e6, 
            (rx**2)/1e9,  (rx*ry)/1e9,     (ry**2)/1e9,
            (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12,    (ry**3)/1e12,

        ]).T

        X_dy = np.vstack([
            np.ones(len(ry)), 
            (ry)/1e6,     (rx)/1e6,
            (ry**2)/1e9,  (ry*rx)/1e9,     (rx**2)/1e9,
            (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12,    
            
        ]).T
        


        coeff_dx = np.linalg.lstsq(X_dx, Yx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Yy, rcond=None)[0]


        # 회귀분석 결과를 리스트에 저장
        shot_regression_results.append({
            'UNIQUE_ID': unique_id,
            'DieX': die_x,
            'DieY': die_y,
            'RK1': coeff_dx[0],
            'RK2': coeff_dy[0],
            'RK3': coeff_dx[1],
            'RK4': coeff_dy[1],
            'RK5': coeff_dx[2],
            'RK6': coeff_dy[2],

            'RK7': coeff_dx[3],
            'RK8': coeff_dy[3],
            'RK9': coeff_dx[4],
            'RK10': coeff_dy[4],
            'RK11': coeff_dx[5],
            'RK12': coeff_dy[5],
            'RK13': coeff_dx[6],
            'RK14': coeff_dy[6],
            'RK15': coeff_dx[7],
            'RK16': coeff_dy[7],
            'RK17': coeff_dx[8],
            'RK18': coeff_dy[8],
            'RK19': coeff_dx[9],
            'RK20': 0,            
        })
    
    # 회귀분석 결과를 새로운 DataFrame으로 변환
    df_cpe19p = pd.DataFrame(shot_regression_results)
    return df_cpe19p





###################### CPE 19para을 fitting ################################ 

def cpe19p_fitting(df_rawdata, df_cpe19p):
    # 결과를 저장할 리스트 생성
    predictions_list = []

    # 각 데이터 포인트에 대해 처리
    for idx, row in df_rawdata.iterrows():
        
        unique_id = row['UNIQUE_ID']
        die_x = row['DieX']
        die_y = row['DieY']
        coordinate_x = row['coordinate_X']
        coordinate_y = row['coordinate_Y']
        residual_x_depsm = row['residual_x_depsm']
        residual_y_depsm = row['residual_y_depsm']
        
        rx = coordinate_x
        ry = coordinate_y

        # 해당 LOT_ID, DieX, DieY에 해당하는 회귀계수 찾기
        coeffs = df_cpe19p[(df_cpe19p['UNIQUE_ID'] == unique_id) & (df_cpe19p['DieX'] == die_x) & (df_cpe19p['DieY'] == die_y)]
        
        if not coeffs.empty:
            coeffs = coeffs.iloc[0]
            # 회귀계수 추출
            RK1 = coeffs['RK1']
            RK2 = coeffs['RK2']
            RK3 = coeffs['RK3']
            RK4 = coeffs['RK4']
            RK5 = coeffs['RK5']
            RK6 = coeffs['RK6']
            RK7 = coeffs['RK7']
            RK8 = coeffs['RK8']
            RK9 = coeffs['RK9']
            RK10 = coeffs['RK10']
            RK11 = coeffs['RK11']
            RK12 = coeffs['RK12']
            RK13 = coeffs['RK13']
            RK14 = coeffs['RK14']
            RK15 = coeffs['RK15']
            RK16 = coeffs['RK16']
            RK17 = coeffs['RK17']
            RK18 = coeffs['RK18']
            RK19 = coeffs['RK19']
            RK20 = coeffs['RK20']

                        
            
            # 예측값 계산
            cpe19p_pred_x = (RK1 
            + RK3*(rx/1e6) + RK5*(-ry/1e6) 
            + RK7*(rx**2)/1e9 + RK9*(rx*ry)/1e9 + RK11*(ry**2)/1e9 
            + RK13*(rx**3)/1e12 + RK15*(rx**2*ry)/1e12 + RK17*(rx*ry**2)/1e12 + RK19*(ry**3)/1e12 )
            

            cpe19p_pred_y = (RK2 
            + RK4*(ry/1e6) + RK6*(rx/1e6)
            + RK8*(ry**2)/1e9 + RK10*(ry*rx)/1e9 + RK12*(rx**2)/1e9
            + RK14*(ry**3)/1e12 + RK16*(ry**2*rx)/1e12 + RK18*(ry*rx**2)/1e12 + RK20*(rx**3)/1e12 )

            


            
            # CPE 잔차값 계산
            cpe19p_resi_x = residual_x_depsm - cpe19p_pred_x 
            cpe19p_resi_y = residual_y_depsm - cpe19p_pred_y

            # 결과 저장
            predictions_list.append({
                'cpe19p_pred_x': cpe19p_pred_x,
                'cpe19p_pred_y': cpe19p_pred_y,
                'cpe19p_resi_x': cpe19p_resi_x,
                'cpe19p_resi_y': cpe19p_resi_y
            })
        else:
            # 해당하는 회귀계수가 없을 경우 NaN 처리
            predictions_list.append({
                'cpe19p_pred_x': np.nan,
                'cpe19p_pred_y': np.nan,
                'cpe19p_resi_x': np.nan,
                'cpe19p_resi_y': np.nan
            })
    
    # 결과를 DataFrame으로 변환
    df_cpe19p_fit_res = pd.DataFrame(predictions_list)

    return df_cpe19p_fit_res






###################### CPE 19para fitting을 부호반대처리 (=Ideal PSM) ################################ 

def ideal_psm(df_rawdata):
    df_rawdata['ideal_psm_x'] = 0- df_rawdata['cpe19p_pred_x']
    df_rawdata['ideal_psm_y'] = 0- df_rawdata['cpe19p_pred_y']

    return df_rawdata





################################### DELTA PSM  #####################################################################

def delta_psm(df_rawdata):
    df_rawdata['delta_psm_x'] = df_rawdata['ideal_psm_x'] - df_rawdata['psm_fit_x']
    df_rawdata['delta_psm_y'] = df_rawdata['ideal_psm_y'] - df_rawdata['psm_fit_y']

    return df_rawdata
























################################### MRC Decorrect #####################################################################

# 데이타 불러오기 


df_rawdata = pd.read_csv("RawData-1.csv")
df_mrc_input = pd.read_csv("MRC.csv")


# mrc k값을 fitting하고 X_reg에서 빼주기.  
df_mrc_de = kmrc_decorrect(df_rawdata, df_mrc_input)
df_rawdata = pd.concat([df_rawdata, df_mrc_de], axis=1)  # axis=1은 열 단위로 병합하는 것을 의미 (행 단위 병합은 axis=0)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'MRC Decorrect 완료.')


################################### raw calc #####################################################################


# X_reg_demrc + MRC X 
df_rawdata = remove_psm_add_pointmrc(df_rawdata)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'Raw 처리(raw_x = X_reg_demrc + MRC_X) 완료.')


################################### MULTI LOT REGRESSION #####################################################################


# 회귀분석(WK1023, RK1023/511) 후 결과 저장(.csv)
df_coeff = multi_lot_regression(df_rawdata)
df_coeff.to_csv('OSR_K.csv', index=False)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'OSR Regression 완료.')

################################### MULTI LOT FITTING, RESIDUAL #####################################################################

# 잔차 계산
df_predictions = multi_lot_fitting_residual(df_rawdata, df_coeff)
df_rawdata = pd.concat([df_rawdata, df_predictions], axis=1)  # axis=1은 열 단위로 병합하는 것을 의미 (행 단위 병합은 axis=0)
    
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'OSR Fitting, Residual 완료.')

################################### PSM Input Decorrect #####################################################################

# 데이타 불러오기 
# df_rawdata는 그대로 사용하면 됨. ( 엑셀 안 불러와도 됨. 시간절약)
df_psm_input = pd.read_csv('PerShotMRC.csv')

# PSM Input(Shot별 RK값)을 fitting하고 residual_x에서 빼주기 
df_psm_de = psm_decorrect(df_rawdata, df_psm_input)


##### df_psm_de의 인덱스 리셋 #####
# ★★★ concat쓰려면 인덱스 리셋해야함. df_rawdata의 index와 df_psm_de의 index를 기준으로 병합함. 

df_psm_de = df_psm_de.reset_index(drop=True)
df_rawdata = pd.concat([df_rawdata, df_psm_de], axis=1)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'PSM Decorrect 완료.' )

################################### Resi_depsm을 CPE 19para 모델링 #####################################################################


# CPE 19para 계산 및 저장 (엑셀 -> CSV)
df_cpe19p = resi_to_cpe(df_rawdata)
df_cpe19p.to_csv('CPE_19p.csv', index=False)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'),'CPE 19para Regression 완료.')

################################### CPE 19para를 Fitting #####################################################################

# multi_lot_cpe_fitting 함수 사용하여 예측값 계산
df_cpe19p_fit_res = cpe19p_fitting(df_rawdata, df_cpe19p)
df_rawdata = pd.concat([df_rawdata, df_cpe19p_fit_res], axis=1)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'CPE_38para Fitting, Residual 완료.')

################################### CPE 19para Fit값을 부호반대처리한게 Ideal PSM  #####################################################################

df_ideal_psm = ideal_psm(df_rawdata)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'Ideal PSM 완료.')

################################### DELTA PSM  #####################################################################

df_delta_psm = delta_psm(df_rawdata)


# Delta PSM 결과 저장 (엑셀 -> CSV)
df_rawdata.to_csv('Delta_PSM.csv', index=False)

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'Delta PSM 완료.')





print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '모든 작업이 완료')




2024-11-30 03:02:58 MRC Decorrect 완료.
2024-11-30 03:02:58 Raw 처리(raw_x = X_reg_demrc + MRC_X) 완료.
2024-11-30 03:02:58 OSR Regression 완료.
2024-11-30 03:02:58 OSR Fitting, Residual 완료.
2024-11-30 03:02:58 PSM Decorrect 완료.
2024-11-30 03:02:58 CPE 19para Regression 완료.
2024-11-30 03:02:59 CPE_38para Fitting, Residual 완료.
2024-11-30 03:02:59 Ideal PSM 완료.
2024-11-30 03:02:59 Delta PSM 완료.
2024-11-30 03:02:59 모든 작업이 완료
